## Einfache Datenverarbeitung

Zunächst importieren wir `pandas`. Eine Bibliothek für den leichten Umgang mit tabellarischen Daten aller Art.

Weitere Informationen und hilfreiche Beispiele finden sich unter https://pandas.pydata.org/docs/index.html

In [ ]:
import pandas as pd

### csv-Dateien einlesen

Nehmen wir an unsere Datei liegt in einem Ordner "Daten" im gleichen Verzeichnis wie dieses Notebook und heißt "_01_11_2024.csv". 

Somit ist der relative Pfad f zu dieser Datei: 

In [ ]:
f = 'Daten\\_01_11_2024.csv'

Um diese in einen `pandas.dataframe` einzulesen verwenden wir `pandas.read_csv()`

In [ ]:
df = pd.read_csv(f, sep='\t', parse_dates=[0], date_format='%d.%m.%y %H:%M:%S', decimal=',')

Hierbei sind einige Angaben wichtig: 

- `sep=` gibt an mit welchem Zeichen die Spalten der csv getrennt sind. Typische Möglichkeiten sind Tabstopp `'\t'`, Komma `','` oder Leerzeichen `' '`
- `parse_dates=` gibt an, in welcher Spalte sich Zeitstempel befinden 
- `date_foramt=` gibt das Format der Zeitstempel an 
- `decimal=` gibt das Dezimaltrennzeichen an. In Programmiersprachen ist dies üblicherweise `'.'`, weshalb es wichtig ist, dies für unsere Daten anzugeben

Um zu prüfen, ob beim Einlesen alles geklappt hat, stellt `pandas` die `dataframe.head()` und `dataframe.tail()` Methoden bereit. Diese geben jeweils die ersten bzw. letzten 5 Zeilen des Dataframe zurück

In [ ]:
df.head()

In [ ]:
df.tail()

### Mehrere csv-Dateien einlesen und zusammenfügen


Nun wollen wir aber häufig nicht bloß eine einzelne csv-Datei einlesen, sondern Versuche laufen gerne über einige Tage und somit müssen wir mehrere Dateien verknüpfen. 

Der einfachste Weg hierfür ist die Pfade der einzelnen Dateien in eine Liste zu schreiben

In [ ]:
f_list = ['Daten\\_05_11_2024.csv', 'Daten\\_02_11_2024.csv', 'Daten\\_03_11_2024.csv', 'Daten\\_04_11_2024.csv', 'Daten\\_01_11_2024.csv']

_Das Ganze geht natürlich noch deutlich cleverer indem man zum Beispiel alle Dateien in einem Verzeichnis einließt, wie dies mit dem folgenden Code möglich ist._

In [ ]:
from os import listdir
from os.path import isfile, join


mypath = 'Daten/'
f_list = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

  
  
  
Mit der Liste an Dateipfaden lesen wir jetzt zunächst alle csv-Dateien ein und legen sie ebenfalls in einer Liste ab. 

Hierfür verwenden wir wieder `pandas.read_csv()`. Verschachteln es aber diesmal mit einer `for`-Schleife und hängen die einzelnen `dataframes` an die zuvor erstellte Liste `df_list` mittels `list.append()`an. 

In [ ]:
df_list = []
for f in f_list: 
    df_list.append(pd.read_csv(f, sep='\t', parse_dates=[0], date_format='%d.%m.%y %H:%M:%S', decimal=','))

Aus dieser Liste machen wir jetzt mit der Funktion `pandas.concat()` einen großen `dataframe`.

In [ ]:
df_full = pd.concat(df_list, ignore_index=True) 

Um zu prüfen, ob das geklappt hat können wir wieder die `head()` und `tail()` Methoden nutzen. 

So sollte jetzt der Anfang des `dataframe` an einem Tag sein und das Ende an einem anderen. Keine Sorge, falls dies noch nicht der erste und letzte Tag der Messung sind. Um die zeitliche Sortierung kümmern wir uns im nächsten Schritt. 

In [ ]:
df_full.head()

In [ ]:
df_full.tail()

### Sortieren des Datensatzes

Die Messreihe sollte idealerweise in einer logischen zeitlichen Abfolge liegen. Je nach Namensgebung der Dateien, kann das aber auch mal besser oder schlechter direkt klappen. Um trotzdem alle Daten in der richtigen Reihenfogle zu haben, zeigen wir jetzt mal das Sortieren der Daten am Beispiel der Timestamps in der Spalte 'Zeit' mittels `dataframe.sort_values()` und im Anschluss setzen wir die Timestamps auch direkt als Index für den Datensatz mittels `dataframe.set_index()`. 

Zur Überprüfung dient erneut `df.head()`

In [ ]:
df_sorted = df_full.sort_values(['Zeit'])
df_time_indexed = df_sorted.set_index('Zeit')

df_time_indexed.head()

_Man kann natürlich auch nach allen möglichen anderen Spalten sortieren, wenn man das möchte._

### Weiterverarbeitung des Datensatzes

#### Bilden eines Mittelwertes über mehrere Sensoren

Um nun mit der tatsächlichen Auswertung unserer Daten zu beginnen, haben wir uns vom Teststandsbetreuer sagen lassen, dass die Thermoelemente TC02, TC03, TC04 und TC05 alle in der relevanten Zone liegen und es hilfreich ist, den Mittelwert der Temperatur über diese zu bilden. Diesen neuen Wert möchten wir natürlich erstellen und ebenfalls in unserem Datensatz als Spalte 'Mittlere Katalysatortempeartur' abspeichern. 

Hierfür bilden wir zunächst eine Liste mit den Spaltennamen aller Thermoelemente über die wir mitteln wollen

In [ ]:
TC_list = ['TC02', 'TC03', 'TC04', 'TC05']

und erstellen dann eine neue Spalte in unserem `dataframe` mit dem gewünschten Namen, in welche wir dann den Mittelwert hineinschreiben, welchen wir mittels `dataframe.mean()` ermitteln. Die zuvor erstellte Liste aus Spaltennamen der Thermoelemente dient uns dabei als Eingrenzung, sodass nur über diese Spalten gemittelt wird. Das Argument `axis=1` gibt dem Code darüber hianus zu verstehen, dass wir zeilenweise mitteln wollen und nicht spaltenweise (`axis=0`) 

In [ ]:
df_time_indexed['Mittlere Katalysatortemperatur'] = df_time_indexed[TC_list].mean(axis=1)
df_time_indexed.head()

`df_time_indexed['Mittlere Katalysatortemperatur'] = df_time_indexed[['TC02', 'TC03', 'TC04', 'TC05']].mean(axis=1)`  
_funktioniert natürlich genauso, ist aber weniger lesbar_

### Reduktion der Daten mittels Resampling

Im Gespräch mit dem Teststandsverantworlichen ist klar geworden, dass der Testverlauf viel träger war als erwartet und die hohe Messfrequenz von 0,5 Hz gar nicht notwendig war. Um den Umgang mit den Daten zu vereinfachen können wir die Daten also reduzieren indem wir für alle Zeilen Mittelwerte über mehere Zeitschritte bilden. Als Startpunkt hierfür wählen wir zunächst 1 Minute. 

Auch hierfür hält `pandas` eine Funktion bereit. Mittels `dataframe.resample()` lassen sich Daten zu Gruppen zusammenfügen und dann wie zuvor mitteln (nur jetzt natürlich spaltenweise mittels `axis=0`)

In [ ]:
window = '5min'
df_resampled = df_time_indexed.resample(window).mean()

df_resampled.head()

Mit der gleichen Gruppierungsfunktion lässt sich auch die Standardabweichung eines Sensors über einen gewissen Zeitraum sehr leicht bestimmen. 

In [ ]:
df_resampled['STD NH3(%)'] = df_time_indexed['NH3 (%)'].resample(window).std()
df_resampled.head()

### Abspeichern der Daten

Nachdem wir nun ein wenig mit unseren Daten gearbeitet haben, ist es sicherlich hilfreich diese abzuspeichern, um nicht jedes Mal alle vorherigen Schritte wieder ausführen zu müssen. Hierfür bieten `dataframe` Objekte die Methode `to_csv()`, die sehr ähnlich zu `read_csv`weiter oben funktioniert. 

In [ ]:
df_resampled.to_csv('Daten\Full.csv', sep='\t', date_format='%d.%m.%y %H:%M:%S', decimal=',')

### Einfaches Plotten von Daten

Nachdem wir unsere Daten nun hinreichend weiterverarbeitet haben, wollen wir noch schnell einige einfache Plots anfertigen. 

Zunächst können wir den am MFC 03.20 gemessenen Volumenstrom über den Verlauf der Messung plotten. Hierbei wollen wir zunächst erst einmal schauen, ob der Versuch tatsächlich wie geplant durchgelaufen ist, daher kümmern wir uns noch nicht um Feinheiten, wie Achsenbeschriftungen oder ähnliches. Für solche schnellen Plots eignet sich `dataframe.plot()`

In [ ]:
df_resampled[['03.20 (MFC03)']].plot()

Als nächstes wollen wir nun den im Produktgases gemessenen Ammoniakgehalt im Verlauf der Messung auftragen. Da diese Grafik später in einer Präsentation landen soll, sollten wir auf korrekte Beschriftungen Acht geben. 

Hierfür fügen wir `plot()` noch einige Argumente hinzu
- `title`: Der Titel des Diagramms
- `ylabel`: Die Beschriftung der y-Achse
- `ylim`: Das obere und untere Limit der y-Achse


In [ ]:
df_resampled[['NH3 (%)']].plot(title='Verlauf des Ammoniakgehalts im Produktgas', ylabel='NH3-Anteil [mol-%]', ylim=(0,120))

_Man kann auch mehrere Linien in einem Diagramm plotten, indem man statt nur eines Spaltennamens mehrere Namen angibt_

In [ ]:
df_resampled[['03.20 (MFC03)', 'Mittlere Katalysatortemperatur']].plot(ylabel='Temepratur [°C] / Volumenstrom [ml/min]')

_Auch das erstellen von Subplots untereinander ist sehr einfach über das_ `subplots` _Argument von_ `dataframe.plot()` _möglich_

In [ ]:
df_resampled[['03.20 (MFC03)', 'Mittlere Katalysatortemperatur', 'NH3 (%)']].plot(subplots=[('03.20 (MFC03)', 'Mittlere Katalysatortemperatur')])

_Mit ein bischen komplexerem Code und der direkten Nutzung von `matplotlib` lassen sich diese Diagramme auch zu einem Diagramm mit zwei y-Achsen zusammenführen_

In [ ]:
import matplotlib.pyplot as plt

ax = df_resampled[['03.20 (MFC03)', 'Mittlere Katalysatortemperatur']].plot(ylabel = 'Temperatur [°C] / Volumenstrom [ml/min]', ylim=(0,2000))
df_resampled[['NH3 (%)']].plot(ax=ax,secondary_y = True, ylabel = 'NH3-Anteil [vol-%]', ylim=(0,130))
plt.show()

Dies kann man jetzt noch beliebig lange fortsetzen. Der Kreativität sind bei der Datendarstellung nahezu keine Grenzen gesetzt.  
Generell sind hier noch einige hilfreiche Bibliotheken zu erwähnen: 
- `seaborne`: Hiermit lassen sich schnell schöne (visuell ansprechende) Plots erstellen, allerdings ist die mögliche Komplexität der Plots begrenzt. 
- `plotly`: Hiermit lassen sich gut interaktive Plots erstellen, in denen man herumscrollen und Datenreiehn zueinander verscheiben kann, allerdings ist auch hier der Funktionsumfang auf gängige Plottypen begrenzt
- `matplotlib`: Das ist älteste Plotting Bibliothek in Python. Sie kann nahezu Alles, was andere Bibliotheken können, aber bis ein Plot nicht nur nützlich sondern auch schön ist, können einige Zeilen Code vergehen